# Webscraping 

Web scraping the Wine Enthusiest website "winemag.com" for oregon wine reviews and wine information.

Started with Zoey's Original code. She found many, many reviews.



In [6]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import requests


In [13]:
response = requests.get("https://www.winemag.com/?s=oregon%20wine&drink_type=wine&page=3&search_type=reviews")

In [14]:
response

<Response [403]>

In [5]:
executable_path = {'executable_path': ChromeDriverManager().install()}

In [16]:
browser = Browser('chrome', **executable_path, headless=False)

In [17]:
browser.visit("https://www.winemag.com/?s=oregon%20wine&drink_type=wine&page=3&search_type=reviews")

In [19]:
soup = bs(browser.html, 'html.parser')
soup

<html class="win chrome chrome1 webkit webkit5 js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths webp webp-alpha webp-animation webp-lossless" lang="en-US"><head><meta class="foundation-mq-small"/><meta class="foundation-mq-small-only"/><meta class="foundation-mq-medium"/><meta class="foundation-mq-medium-only"/><meta class="foundation-mq-large"/><meta class="foundation-mq-large-only"/><meta class="foundation-mq-xlarge"/><meta class="foundation-mq-xlarge-only"/><meta class="foundation-mq-xxlarge"/><meta class="foundation-data-attribute-namespace"/>
<meta charset="utf-8"/>
<met

In [26]:
li = soup.find_all("li", class_="review-item")
li

[<li class="review-item">
 <a class="review-listing row" data-review-id="381767" href="https://www.winemag.com/buying-guide/abacela-2018-grenache-umpqua-valley/">
 <div class="show-for-small-only heading-wrapper columns small-16">
 <img alt="review label" class="listing-label-mobile hide" src="https://www.winemag.com/wp-content/assets/reviews/label-images/wine/Wine_Default_label.jpg"/>
 <h3 class="title">Abacela 2018 Grenache (Umpqua Valley)</h3>
 <span class="appellation">Southern Oregon</span>
 </div>
 <section class="columns medium-14 small-12 heading-wrapper">
 <div class="hide-for-small">
 <img alt="review label" class="listing-label hide" src="https://www.winemag.com/wp-content/assets/reviews/label-images/wine/Wine_Default_label.jpg"/>
 <h3 class="title">Abacela 2018 Grenache (Umpqua Valley)</h3>
 <span class="appellation">Southern Oregon</span>
 </div>
 <div class="excerpt">Grenache is carving out an important role in southern Oregon. Here it ...</div>
 <div class="badge-wrapper

In [59]:
# Grabbing the search information in the results 
cellar = []
for tag in li[:2]:
    wine = tag.find("div", class_="hide-for-small")
    price = tag.find("span", class_="price").text
    rating = tag.find("span", class_="rating").text
    full_review_link = tag.find("a").get("href")
    bottle_name = wine.find("h3").text
    region = wine.find("span", class_="appellation").text
    cellar.append({"price":price, "rating":rating, "bottle_name":bottle_name, "region":region})
    browser.visit(full_review_link)
    sleep(30)
cellar    

[{'price': '$30',
  'rating': '91 Points',
  'bottle_name': 'Abacela 2018 Grenache (Umpqua Valley)',
  'region': 'Southern Oregon'},
 {'price': '$32',
  'rating': '90 Points',
  'bottle_name': 'Valcan 2017 El Torero Tempranillo (Rogue Valley)',
  'region': 'Southern Oregon'}]

In [60]:
browser.visit(full_review_link)

In [102]:
## Grabbing information from the full review page
soup=bs(browser.html, "html.parser")
section = soup.find("section", id="review")
section.find("div", class_="header__title").text
section.find("p", class_="description").text
ul = section.find("div", class_="info-section").find_all("ul")
for tag in ul:
    for li in tag.find_all('li'):
#         label = 
        print(li.find("div", class_="info-label").text.replace("\n", ""))
        print(li.find("div", class_="info").text.replace("\n", "").split(','))
        print("===========\n=============\n========")

rating
['90']
Price
['$32', 'Buy Now']
Designation
['El Torero']
Variety
['Tempranillo']
Appellation
['Rogue Valley', ' Southern Oregon', ' Oregon', ' US']
Winery
['Valcan']
Alcohol
['14%']
Bottle Size
['750 ml']
Category
['Red']
Date Published
['12/1/2021']
User Avg Rating
['Not rated yet [Add Your Review]']


In [103]:
browser.visit(f"https://www.winemag.com/?s=oregon%20wine&drink_type=wine&page=1&search_type=reviews")

In [105]:
## Find the number of paes to loop through
soup = bs(browser.html, "html.parser")

In [111]:
int(soup.find("div", class_="pagination").find_all('li')[-1].text.replace('\n', ""))

38

In [117]:
## put it all together
cellar = []

## visit the first page
browser.visit(f"https://www.winemag.com/?s=oregon%20wine&drink_type=wine&page=1&search_type=reviews")
last_page = int(soup.find("div", class_="pagination").find_all('li')[-1].text.replace('\n', ""))
## Loop through all the pages
for i in range(1, (last_page+1)):
    if i >1:
        browser.visit(f"https://www.winemag.com/?s=oregon%20wine&drink_type=wine&page={i}&search_type=reviews")
        sleep(5)
    else:
        pass
    soup = bs(browser.html, 'html.parser')
    li = soup.find_all("li", class_="review-item")
    # Grabbing the search information in the results 
    for tag in li:
        wine = tag.find("div", class_="hide-for-small")
        price = tag.find("span", class_="price").text
        rating = tag.find("span", class_="rating").text
        full_review_link = tag.find("a").get("href")
        bottle_name = wine.find("h3").text
        region = wine.find("span", class_="appellation").text
        bottle = {"price":price, "rating":rating, "bottle_name":bottle_name, "region":region,
                  "review_link":full_review_link, "source":"Wine Enthusiast"}

    ## Grabbing information from the full review page
        browser.visit(full_review_link)
        review=bs(browser.html, "html.parser")
        section = review.find("section", id="review")
        section.find("div", class_="header__title").text
        review_text = section.find("p", class_="description").text
        ul = section.find("div", class_="info-section").find_all("ul")
        for tag1 in ul:
            for info in tag1.find_all('li'):
                key = info.find("div", class_="info-label").text.replace("\n", "")
                bottle[key] = info.find("div", class_="info").text.replace("\n", "").split(',')
                
        cellar.append(bottle)        
        sleep(10)


In [118]:
len(cellar)

741

In [141]:
enthusiast_df = pd.DataFrame(cellar)
enthusiast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   price            741 non-null    object
 1   rating           741 non-null    object
 2   bottle_name      741 non-null    object
 3   region           741 non-null    object
 4   review_link      741 non-null    object
 5   source           741 non-null    object
 6   Price            741 non-null    object
 7   Designation      427 non-null    object
 8   Variety          741 non-null    object
 9   Appellation      741 non-null    object
 10  Winery           741 non-null    object
 11  Alcohol          741 non-null    object
 12  Bottle Size      741 non-null    object
 13  Category         741 non-null    object
 14  Date Published   741 non-null    object
 15  User Avg Rating  741 non-null    object
 16  Importer         4 non-null      object
dtypes: object(17)
memory usage: 98.5+ K

In [146]:
# enthusiast_df["rating"] = enthusiast_df["rating"].apply(lambda x: x[0])
# enthusiast_df["price"] = enthusiast_df["price"].apply(lambda x: x[:3])
enthusiast_df.to_csv("raw_wine_enthusiast_reviews.csv")

In [1]:
import pandas as pd
df = pd.read_csv("raw_wine_enthusiast_reviews.csv")

In [2]:
df

,Unnamed: 0,price,rating,bottle_name,region,review_link,source,Price,Designation,Variety,Appellation,Winery,Alcohol,Bottle Size,Category,Date Published,User Avg Rating,Importer
0,0,$13,88,The Great Oregon Wine Company 2021 Rascal Pino...,Oregon Other,https://www.winemag.com/buying-guide/the-great...,Wine Enthusiast,"['$13', 'Buy Now']",['Rascal'],['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['The Great Oregon Wine Company'],['13.3%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
1,1,$19,86,Wine by Joe 2019 Pinot Noir (Oregon),Oregon Other,https://www.winemag.com/buying-guide/wine-by-j...,Wine Enthusiast,"['$19', 'Buy Now']",NaN,['Pinot Noir'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Wine by Joe'],['13.5%'],['750 ml'],['Red'],['10/1/2022'],['Not rated yet [Add Your Review]'],NaN
2,2,$15,88,Duck Pond Cellars 2021 Oregon Wine Co. Chardon...,Oregon Other,https://www.winemag.com/buying-guide/duck-pond...,Wine Enthusiast,"['$15', 'Buy Now']",['Oregon Wine Co.'],['Chardonnay'],"['Oregon', ' Oregon Other', ' Oregon', ' US']",['Duck Pond Cellars'],['13.2%'],['750 ml'],['White'],['9/1/2022'],['Not rated yet [Add Your Review]'],NaN
3,3,$75,92,Domaine Drouhin 2019 Laurene Pinot Noir (Dunde...,Willamette Valley,https://www.winemag.com/buying-guide/domaine-d...,Wine Enthusiast,"['$75', 'Buy Now']",['Laurene'],['Pinot Noir'],"['Dundee Hills', ' Willamette Valley', ' Orego...",['Domaine Drouhin'],['13.9%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
4,4,$40,90,Portlandia 2019 Momtazi Vineyard Pinot Noir (W...,Willamette Valley,https://www.winemag.com/buying-guide/portlandi...,Wine Enthusiast,"['$40', 'Buy Now']",['Momtazi Vineyard'],['Pinot Noir'],"['Willamette Valley', ' Willamette Valley', ' ...",['Portlandia'],['13.5%'],['750 ml'],['Red'],['7/1/2022'],['Not rated yet [Add Your Review]'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,736,$18,90,Ponzi 1998 Arneis (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/ponzi-199...,Wine Enthusiast,"['$18', 'Buy Now']",NaN,"['Arneis', ' Italian White']","['Willamette Valley', ' Willamette Valley', ' ...",['Ponzi'],['N/A'],['500 ml'],['White'],['4/1/2000'],['Not rated yet [Add Your Review]'],NaN
737,737,$12,86,Tyee 1996 Pinot Blanc (Willamette Valley),Willamette Valley,https://www.winemag.com/buying-guide/tyee-1996...,Wine Enthusiast,"['$12', 'Buy Now']",NaN,['Pinot Blanc'],"['Willamette Valley', ' Willamette Valley', ' ...",['Tyee'],['13%'],['750 ml'],['White'],['12/31/1999'],['Not rated yet [Add Your Review]'],NaN
738,738,$35,92,Cameron 1997 Clos Electrique Chardonnay (Willa...,Willamette Valley,https://www.winemag.com/buying-guide/cameron-1...,Wine Enthusiast,"['$35', 'Buy Now']",['Clos Electrique'],['Chardonnay'],"['Willamette Valley', ' Willamette Valley', ' ...",['Cameron'],['12.7%'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN
739,739,$15,90,Torii Mor 1998 Pinot Gris (Yamhill County),Willamette Valley,https://www.winemag.com/buying-guide/torii-mor...,Wine Enthusiast,"['$15', 'Buy Now']",NaN,['Pinot Gris'],"['Yamhill County', ' Willamette Valley', ' Ore...",['Torii Mor'],['N/A'],['750 ml'],['White'],['11/1/1999'],['Not rated yet [Add Your Review]'],NaN
